In [1]:
# ! pip install git+https://github.com/modestyachts/ImageNetV2_pytorch

from imagenetv2_pytorch import ImageNetV2Dataset
import clip
from dataset import VLImageDataset
import torch
from tqdm import tqdm
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'.')
import torch
import torch.nn as nn
from tqdm import tqdm
import argparse
from importlib import reload,import_module
import multiprocessing
import os
import time
from itertools import product
sys.path.append("./PTQ4ViT")
import PTQ4ViT.utils.datasets as datasets
import PTQ4ViT.utils.net_wrap as net_wrap
from PTQ4ViT.utils.quant_calib import QuantCalibrator, HessianQuantCalibrator
from PTQ4ViT.utils.models import get_net
from dataset import VLImageDataset, VLLoader


['.', '..', '/nfs/hpc/share/pullelas/DL-Project', '', '/usr/local/apps/anaconda/2023.03/lib/python3.9/site-packages', '/usr/local/apps/anaconda/2023.03/lib/python310.zip', '/usr/local/apps/anaconda/2023.03/lib/python3.10', '/usr/local/apps/anaconda/2023.03/lib/python3.10/lib-dynload', '/nfs/stak/users/pullelas/.local/lib/python3.10/site-packages', '/nfs/hpc/share/pullelas/DL-Project/CLIP', '/usr/local/apps/anaconda/2023.03/lib/python3.10/site-packages', '/usr/local/apps/anaconda/2023.03/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg', '/usr/local/apps/anaconda/2023.03/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg', './PTQ4ViT', './PTQ4ViT/utils']


In [2]:
# !pip install timm

In [3]:
model, preprocess = clip.load("ViT-B/32", device="cuda")


In [4]:
def test_classification(net,test_loader,max_iteration=None, description=None):
    pos=0
    tot=0
    i = 0
    max_iteration = len(test_loader) if max_iteration is None else max_iteration
    with torch.no_grad():
        q=tqdm(test_loader, desc=description)
        for inp,target in q:
            i+=1
            inp=inp.cuda()
            target=target.cuda()
            out=net(inp)
            pos_num=torch.sum(out.argmax(1)==target).item()
            pos+=pos_num
            tot+=inp.size(0)
            q.set_postfix({"acc":pos/tot})
            if i >= max_iteration:
                break
    print(pos/tot)
    return pos/tot

In [5]:
def init_config(config_name):
    """initialize the config. Use reload to make sure it's fresh one!"""
    _,_,files =  next(os.walk("./PTQ4ViT/configs"))
    if config_name+".py" in files:
        quant_cfg = import_module(f"configs.{config_name}")
    else:
        raise NotImplementedError(f"Invalid config name {config_name}")
    reload(quant_cfg)
    return quant_cfg

In [11]:
def experiment_basic(net='vit_base_patch16_384', config="PTQ4ViT"):
    """
    A basic testbench.
    """
    quant_cfg = init_config(config)
    net = model
    #print("before:",net.eval())
    wrapped_modules = net_wrap.wrap_modules_in_net(net,quant_cfg)
    #print("after:",net.eval())
    # g=datasets.ViTImageNetLoaderGenerator('imagenetv2-matched-frequency-format-val','imagenet',32,32,16,kwargs={"model":net})
    # test_loader=g.test_loader()
    # calib_loader=g.calib_loader(num=32)
    image_features, text_features, target = VLLoader(model, preprocess)
    g = VLImageDataset(image_features.cuda(), text_features.cuda(), target.cuda())
    calib_loader = torch.utils.data.DataLoader(g, batch_size=32)

    quant_calibrator = HessianQuantCalibrator(net,wrapped_modules,calib_loader,sequential=False,batch_size=4) # 16 is too big for ViT-L-16
    quant_calibrator.batching_quant_calib()
    
    # test_classification(net,test_loader)

experiment_basic()

Completed net wrap.
prepare parallel calibration for ['visual.conv1', 'visual.transformer.resblocks.0.attn.out_proj', 'visual.transformer.resblocks.0.mlp.c_fc', 'visual.transformer.resblocks.0.mlp.c_proj', 'visual.transformer.resblocks.1.attn.out_proj', 'visual.transformer.resblocks.1.mlp.c_fc', 'visual.transformer.resblocks.1.mlp.c_proj', 'visual.transformer.resblocks.2.attn.out_proj', 'visual.transformer.resblocks.2.mlp.c_fc', 'visual.transformer.resblocks.2.mlp.c_proj', 'visual.transformer.resblocks.3.attn.out_proj', 'visual.transformer.resblocks.3.mlp.c_fc', 'visual.transformer.resblocks.3.mlp.c_proj', 'visual.transformer.resblocks.4.attn.out_proj', 'visual.transformer.resblocks.4.mlp.c_fc', 'visual.transformer.resblocks.4.mlp.c_proj', 'visual.transformer.resblocks.5.attn.out_proj', 'visual.transformer.resblocks.5.mlp.c_fc', 'visual.transformer.resblocks.5.mlp.c_proj', 'visual.transformer.resblocks.6.attn.out_proj', 'visual.transformer.resblocks.6.mlp.c_fc', 'visual.transformer.res

Hessian:   0%|          | 0/73 [00:00<?, ?it/s, visual.conv1]

visual.conv1 ChannelwiseBatchingQuantConv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)


/nfs/stak/users/pullelas/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
Hessian:   1%|▏         | 1/73 [00:01<01:28,  1.23s/it, visual.transformer.resblocks.0.attn.out_proj]

visual.transformer.resblocks.0.attn.out_proj PTQSLBatchingQuantLinear(in_features=768, out_features=768, bias=True)


Hessian:   1%|▏         | 1/73 [00:01<02:21,  1.97s/it, visual.transformer.resblocks.0.attn.out_proj]

None


TypeError: cat() received an invalid combination of arguments - got (NoneType, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [8]:
for batch_st in range(0,32,4):
    print(batch_st)

0
4
8
12
16
20
24
28


In [9]:
m = nn.Linear(20, 30)

In [10]:
m.rawinput

AttributeError: 'Linear' object has no attribute 'rawinput'